In [2]:
!pip install torchaudio pytorch_lightning

     |████████████████████████████████| 1.9 MB 5.2 MB/s 
     |████████████████████████████████| 925 kB 38.2 MB/s 
     |████████████████████████████████| 831.4 MB 7.3 kB/s 
     |████████████████████████████████| 829 kB 50.5 MB/s 
     |████████████████████████████████| 636 kB 32.5 MB/s 
     |████████████████████████████████| 125 kB 39.0 MB/s 
     |████████████████████████████████| 282 kB 50.7 MB/s 
     |████████████████████████████████| 1.3 MB 38.5 MB/s 
     |████████████████████████████████| 142 kB 43.9 MB/s 
     |████████████████████████████████| 294 kB 46.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=2998e49602fbe99f7445a226802e3f63ece3024525aeb0a5abed0c13940e01e2
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully

In [1]:
import torch, torchaudio
from torch import nn
from torch.nn import functional as F

import pytorch_lightning as pl
from pytorch_lightning.metrics import functional

import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from google.colab import drive


drive.mount('/content/drive')
torch.cuda.is_available()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [4]:
!ls "drive/My Drive/EEGNET"

EEGNET.ipynb  eeg.pkl  features.pkl


In [5]:
path = Path("drive/My Drive/EEGNET")

In [6]:
feat = pickle.load(open("drive/My Drive/EEGNET/features.pkl", "rb"))
eeg = pickle.load(open("drive/My Drive/EEGNET/eeg.pkl", "rb"))

X = torch.from_numpy(feat['X'])
y = torch.from_numpy(feat['y'])
eeg = torch.from_numpy(eeg['EEG'])

In [35]:
class EEGDataset(torch.utils.data.Dataset):
    # Simple class to load the desired folders inside ESC-50
    
    def __init__(self, path: Path = Path("drive/My Drive/EEGNET"), 
                 sample_rate: int = 8000):
        # Load CSV & initialize all torchaudio.transforms:
        # Resample --> MelSpectrogram --> AmplitudeToDB

        feat = pickle.load(open("drive/My Drive/EEGNET/features.pkl", "rb"))
        eeg = pickle.load(open("drive/My Drive/EEGNET/eeg.pkl", "rb"))

        self.X = torch.from_numpy(feat['X']).float()
        self.y = torch.from_numpy(feat['y']).float()
        self.eeg = torch.from_numpy(eeg['EEG']).float()

        self.resample = torchaudio.transforms.Resample(
            orig_freq=250, new_freq=sample_rate
        ) #useful?
        self.melspec = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate)
        self.db = torchaudio.transforms.AmplitudeToDB(top_db=80)
        
        
    def __getitem__(self, index):
        # Returns (xb, yb) pair, after applying all transformations on the audio file.
        
        wav = self.eeg[index]
        label = self.y[index]
        """
        tmp = []
        for w in wav:
          tmp.append(self.db(
            self.melspec(
                self.resample(w.reshape(1, -1))
            )
        ))

        xb = torch.vstack(tmp)
        """
        return wav, label
        
    def __len__(self):
        # Returns length
        return len(self.eeg)

In [36]:
train_data = EEGDataset()
for xb, yb in train_data:
    break

In [37]:
xb.shape

torch.Size([32, 7488])

In [38]:
# We use folds 1,2,3 for training, 4 for validation, 5 for testing.
train_data = EEGDataset()
val_data = EEGDataset()
test_data = EEGDataset()

train_loader = \
    torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=2)

In [39]:
class AudioNet(pl.LightningModule):
    
    def __init__(self, n_classes = 1, base_filters = 32):
        super().__init__()
        self.conv1 = nn.Conv2d(32, base_filters, 11, padding=5)
        self.bn1 = nn.BatchNorm2d(base_filters)
        self.conv2 = nn.Conv2d(base_filters, base_filters, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(base_filters)
        self.pool1 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(base_filters, base_filters * 2, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(base_filters * 2)
        self.conv4 = nn.Conv2d(base_filters * 2, base_filters * 4, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(base_filters * 4)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(base_filters * 4, n_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool1(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool2(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = self.fc1(x[:, :, 0, 0])
        return torch.squeeze(x)
    
    def training_step(self, batch, batch_idx):
        # Very simple training loop
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, on_step=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
      return self.validation_step(batch, batch_idx)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [40]:
pl.seed_everything(0)
# Test that the network works on a single mini-batch
audionet = AudioNet()
xb, yb = next(iter(train_loader))
audionet(xb).shape

Global seed set to 0


RuntimeError: ignored

In [27]:
xb.shape

torch.Size([4, 32, 128, 1199])

In [25]:
trainer = pl.Trainer(gpus=1, max_epochs=25)
trainer.fit(audionet, train_loader, val_loader)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name  | Type        | Params
---------------------------------------
0  | conv1 | Conv2d      | 123 K 
1  | bn1   | BatchNorm2d | 64    
2  | conv2 | Conv2d      | 9.2 K 
3  | bn2   | BatchNorm2d | 64    
4  | pool1 | MaxPool2d   | 0     
5  | conv3 | Conv2d      | 18.5 K
6  | bn3   | BatchNorm2d | 128   
7  | conv4 | Conv2d      | 73.9 K
8  | bn4   | BatchNorm2d | 256   
9  | pool2 | MaxPool2d   | 0     
10 | fc1   | Linear      | 129   
---------------------------------------
226 K     Trainable params
0         Non-trainable params
226 K     Total params
0.905     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning: The number of training samples (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1051: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [20]:
# TODO: implement the test loop.
trainer.test(audionet, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_loss': 2575.774658203125}
--------------------------------------------------------------------------------


[{'val_loss': 2575.774658203125}]